# Notebook setup

This notebook is intended to be used online on the Einstein Toolkit tutorial server, offline as a read-only document. To make all of these work some setting need to be tweaked, which we do in the next cell.

In [3]:
# this allows you to use "cd" in cells to change directories instead of requiring "%cd"
%automagic on
# override IPython's default %%bash to not buffer all output
from IPython.core.magic import register_cell_magic
@register_cell_magic
def bash(line, cell): get_ipython().system(cell)

# this (non-default package) keeps the end of shell output in view
try: import scrolldown
except ModuleNotFoundError: pass

# We are going to install kuibit, a Python package to post-process Cactus simulations.
# We will install kuibit inside the Cactus directory. The main reason for this is to
# have a make easier to uninstall kuibit (you can just remove the Cactus folder). 
import os, sys
os.environ["PYTHONUSERBASE"] = os.environ['HOME'] + "/Cactus/python"
sys.path.insert(1, f"{os.environ['PYTHONUSERBASE']}/lib/python{sys.version_info[0]}.{sys.version_info[1]}/site-packages")


Automagic is ON, % prefix IS NOT needed for line magics.


# Trying out one of my configurations

The configuration i want to run is the evolution of a TOV star in a cartesian grid in the einstein toolkit environment. This work is taken from my thesis, so let's access the configuration files.
First I'll show where my configuration files are

In [4]:
%%bash
cd ~/programmi/tov_ET/recon_method/
ls | grep .par

tov_ET_eno.par
tov_ET_ppm.par
tov_ET_tvd.par
tov_ET_weno.par


And now i show where my Cactus path is

In [6]:
cd ~/ET/Cactus

/home/sam/ET/Cactus


Now let's run a simple simulation for the weno reconstruction method.
I usually prefer to have my simulations folder inside the ET folder, to do that you need to edit the `basedir` value in `simfactory/etc/defs.local.ini`

``` 
[swift]
basedir = ~/ET/simulations
```

For my case i also decided to move the Cactus directory in another path, if you do so you have to tell the ET that your `sourcebasedir` has changed.
Just go to `/home/sam/ET/Cactus/simfactory/mdb/machines` and look for your machine hostname .ini file and then change the following line, in my case `swift.ini`

```
sourcebasedir   = /home/sam/ET
```

In [7]:
%%bash
./simfactory/bin/sim create tov_ET_weno --configuration sim --parfile=~/programmi/tov_ET/recon_method/tov_ET_weno.par

Parameter file: /home/sam/programmi/tov_ET/recon_method/tov_ET_weno.par
Error: cannot create job skeleton directory: Simulation "/home/sam/ET/simulations/tov_ET_weno" already exists and has been initialized
Aborting Simfactory.


As you can see, the structure of the simulation has been created, that mean ET has allocated the resources and activated the thorns needed in order to run the simulation. One can check the structure of this by going in the directory

In [8]:
%%bash
cd ~/ET/simulations/tov_ET_weno/
ls

log.txt  output-0000  output-0000-active  SIMFACTORY


And finally now, we can start the simulation

In [9]:
%%bash
# start simulation segment
./simfactory/bin/sim submit tov_ET_weno --cores=2 --num-threads=1 --walltime=0:20:00

Assigned restart id: 1 
Executing submit command: exec nohup /home/sam/ET/simulations/tov_ET_weno/output-0001/SIMFACTORY/SubmitScript < /dev/null > /home/sam/ET/simulations/tov_ET_weno/output-0001/tov_ET_weno.out 2> /home/sam/ET/simulations/tov_ET_weno/output-0001/tov_ET_weno.err & echo $!
Submit finished, job id is 389948


The `submit` command submitted a new segment for the simulation `tov_ET_weno` to the queueing system to run in the background asking for a maximum runtime of 20 minutes, using a total of 2 compute cores and using 1 thread per MPI ranks. On your laptop it will start right away, on a cluster the queuing system will wait until a sufficient number of nodes is able to start your simulation.

You can check the status of the simulation with the following command

In [10]:
%%bash
# watch log output, following along as new output is produced
./simfactory/bin/sim show-output --follow tov_ET_weno

Simulation name: tov_ET_weno
The job's Formaline output is:
(file does not exist)
The job's stdout is:
Simulation name: tov_ET_weno
Running simulation tov_ET_weno
Preparing:
Checking:
/home/sam/ET/simulations/tov_ET_weno/output-0001-active
swift
mer 1 mar 2023, 20:06:40, CET
Environment:
Starting:
Simfactory Done at date: mer 1 mar 2023, 20:06:40

The job's stderr is:
+ set -e
+ cd /home/sam/ET/simulations/tov_ET_weno/output-0001-active
+ echo Checking:
+ pwd
+ hostname
+ date
+ echo Environment:
+ export CACTUS_NUM_PROCS=2
+ CACTUS_NUM_PROCS=2
+ export CACTUS_NUM_THREADS=1
+ CACTUS_NUM_THREADS=1
+ export GMON_OUT_PREFIX=gmon.out
+ GMON_OUT_PREFIX=gmon.out
+ export OMP_NUM_THREADS=1
+ OMP_NUM_THREADS=1
+ env
+ sort
+ echo Starting:
++ date +%s
+ export CACTUS_STARTTIME=1677697600
+ CACTUS_STARTTIME=1677697600
+ '[' 2 = 1 ']'
+ mpirun -np 2 /home/sam/ET/simulations/tov_ET_weno/SIMFACTORY/exe/cactus_sim -L 3 /home/sam/ET/simulations/tov_ET_weno/output-0001/tov_ET_weno.par
/home/sam/ET/si

In [11]:
%%bash
./simfactory/bin/sim list-simulations

   helloworld              [ACTIVE (FINISHED), restart 0000, job id -1]
   tov_ET_weno             [ACTIVE (FINISHED), restart 0001, job id 389948]


The output will be stored in a directory, to get its position just run

In [12]:
%%bash
./simfactory/bin/sim get-output-dir tov_ET_weno

/home/sam/ET/simulations/tov_ET_weno/output-0001


If you're done with your simulation just execute

In [13]:
%%bash
./simfactory/bin/sim purge tov_ET_weno

Simulation name: tov_ET_weno
Simulation tov_ET_weno has been moved to trash folder /home/sam/ET/simulations/TRASH/simulation-tov_ET_weno-swift-swift-sam-2023.03.01-20.04.05-389487
